# Environment

In [1]:
# Standard library imports
import os
import shutil
from concurrent.futures import ProcessPoolExecutor, as_completed

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam

# rpy2 imports
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri, r, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.vectors import StrVector, DataFrame

import pandas as pd
import glob
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing

In [2]:
# Set up the working directory
# data_dir = "/beegfs/scratch/ric.broccoli/ric.broccoli/PW_RNA_seq_deep/DexSeq_counts"
data_dir = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords/Create_counts/output"
working_dir = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords"
os.chdir(working_dir)

In [3]:
# Enable automatic conversion between pandas and R dataframes
pandas2ri.activate()

In [4]:
# Import necessary R packages
dexseq = importr('DEXSeq')
deseq2 = importr('DESeq2')

# Local Functions

In [279]:
def compare_file_structures(file_list):
    """
    Compare structures of multiple DEXSeq count files.
    """
    results = []
    for file_path in file_list:
        with open(file_path, 'r') as f:
            n_lines = sum(1 for line in f)
        file_size = os.path.getsize(file_path)
        results.append({
            'file': os.path.basename(file_path),
            'n_lines': n_lines,
            'size_mb': file_size / 1024 / 1024,
            'avg_line_size': file_size / n_lines if n_lines > 0 else 0
        })
    return pd.DataFrame(results)

def parse_dexseq_id(feature_id):
    """Parse DEXSeq feature ID into gene ID and exon number."""
    if feature_id.startswith('*'):
        return feature_id, None
    try:
        # Format is typically 'ENSG00000000003.14:"001"'
        parts = feature_id.split(':')
        gene_id = parts[0]
        exon_num = parts[1].strip('"')
        return gene_id, exon_num
    except:
        return feature_id, None

def examine_dexseq_file(file_path, n_head=5, n_random=5, n_tail=5):
    """
    Examine a DEXSeq count file in detail with corrected parsing and error handling.
    """
    print(f"\nExamining file: {os.path.basename(file_path)}")
    
    # First, peek at the raw file contents
    print("\nFirst few lines (raw):")
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i < 5:
                print(line.strip())
            else:
                break

    # Read the file with correct column interpretation
    try:
        # Read file with custom separator and column names
        df = pd.read_csv(file_path, sep='\s+', header=None,
                        names=['feature_id', 'count'])
        
        # Convert feature_id to string to handle potential float values
        df['feature_id'] = df['feature_id'].astype(str)
        
        # Split feature_id into gene_id and exon_number
        df['gene_id'], df['exon_number'] = zip(*df['feature_id'].apply(parse_dexseq_id))
        
        # File information
        print(f"\nFile size: {os.path.getsize(file_path) / 1024:.2f} KB")
        print(f"Number of lines: {len(df)}")
        
        # Basic DataFrame information
        print("\nDataFrame Info:")
        print(df.info())
        
        # Print all column names
        print("\nColumn names:")
        print(df.columns.tolist())
        
        # Show first few lines
        print(f"\nFirst {n_head} lines:")
        print(df.head(n_head))
        
        # Basic statistics for counts
        print("\nCount statistics:")
        print(df['count'].describe())
        
        # Additional information
        print("\nNumber of unique genes:", df['gene_id'].nunique())
        print("\nTop 5 genes by total counts:")
        gene_counts = df.groupby('gene_id')['count'].sum().sort_values(ascending=False)
        print(gene_counts.head())
        
        # Check for potential issues
        print("\nChecking for potential issues:")
        issues = []
        if df['count'].isnull().any():
            issues.append("- Contains missing values in counts")
        if (df['count'] < 0).any():
            issues.append("- Contains negative counts")
        if not np.issubdtype(df['count'].dtype, np.number):
            issues.append("- Counts are not numeric")
        if df['feature_id'].dtype != object:
            issues.append(f"- Feature ID column is not string type (current type: {df['feature_id'].dtype})")
            
        # Check for special entries
        special_entries = df[df['feature_id'].str.startswith('*')]
        if not special_entries.empty:
            print("\nSpecial entries found:")
            print(special_entries)
            
        if len(issues) > 0:
            print("\nIssues found:")
            print("\n".join(issues))
        else:
            print("No major issues found")
            
        return df
        
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return None

def examine_genes_of_interest(df, genes):
    """
    Examine specific genes in the DEXSeq count data.
    """
    print("\nExamining genes of interest:")
    for gene in genes:
        # Find all entries for this gene
        gene_data = df[df['gene_id'].str.contains(gene, na=False)]
        if not gene_data.empty:
            print(f"\n{gene}:")
            print(f"Number of exons: {len(gene_data)}")
            print(f"Total counts: {gene_data['count'].sum()}")
            print(f"Mean counts per exon: {gene_data['count'].mean():.2f}")
            print("\nExon-level data:")
            print(gene_data.sort_values('exon_number'))
            
            # Create plot for this gene
            plt.figure(figsize=(10, 4))
            plt.bar(range(len(gene_data)), gene_data['count'])
            plt.title(f'Exon counts for {gene}')
            plt.xlabel('Exon number')
            plt.ylabel('Counts')
            plt.tight_layout()
            plt.savefig(f'exon_counts_{gene}.pdf')
            plt.close()

def prepare_sample_table(sample_info):
    """
    Prepare sample table in the format required by DEXSeq
    """
    # Filter for EDO and ND1 samples (excluding PW1)
    samples_subset = sample_info[sample_info['condition'].isin(['EDO', 'ND1'])].copy()
    
    # Reset index and ensure proper categorical variables
    samples_subset = samples_subset.reset_index(drop=True)
    samples_subset['condition'] = pd.Categorical(samples_subset['condition'])
    samples_subset['replicate'] = pd.Categorical(samples_subset['replicate'])
    
    return samples_subset

def create_dexseq_object(sample_table, dexseq):
    """
    Create DEXSeqDataSet object from count files
    """
    # Convert sample table to R dataframe
    with localconverter(ro.default_converter + pandas2ri.converter):
        sample_table_r = ro.conversion.py2rpy(sample_table)
    
    # Create formula for the full model
    formula = Formula('~ sample + exon + condition:exon')
    
    # Create formula for the reduced model
    reduced_formula = Formula('~ sample + exon')
    
    # Create DEXSeqDataSet
    dxd = dexseq.DEXSeqDataSetFromHTSeq(
        countfiles=ro.StrVector(sample_table['count_file']),
        sampleData=sample_table_r,
        design=formula,
        flattenedfile=None  # Set to None if you don't have a flattened annotation file
    )
    
    return dxd

def run_dexseq_analysis(dxd, dexseq):
    """
    Run DEXSeq differential exon usage analysis
    """
    # Normalize counts
    dxd = dexseq.estimateSizeFactors(dxd)
    
    # Estimate dispersions
    dxd = dexseq.estimateDispersions(dxd)
    
    # Test for differential exon usage
    dxd = dexseq.testForDEU(dxd)
    
    # Estimate exon fold changes
    dxd = dexseq.estimateExonFoldChanges(dxd)
    
    return dxd

def extract_results(dxd, dexseq):
    """
    Extract and format DEXSeq results
    """
    # Get results table
    res = dexseq.DEXSeqResults(dxd)
    
    # Convert R results to pandas DataFrame
    with localconverter(ro.default_converter + pandas2ri.converter):
        results_df = ro.conversion.rpy2py(res)
    
    # Add adjusted p-value threshold
    results_df['significant'] = results_df['padj'] < 0.05
    
    return results_df

def filter_significant_results(results_df):
    """
    Filter for significant differential exon usage
    """
    # Filter for adjusted p-value < 0.05
    significant_results = results_df[
        (results_df['padj'] < 0.05) & 
        (results_df['log2fold_EDO_ND1'] != 0)
    ].copy()
    
    # Sort by adjusted p-value
    significant_results = significant_results.sort_values('padj')
    
    return significant_results\

def process_file(file):
    """
    Process a DEXSeq count file and save the formatted version.
    """
    try:
        # Create absolute paths
        output_dir = os.path.abspath(os.path.join(working_dir, 'DexSeq_counts'))
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"Processing file: {file}")
        print(f"Output directory: {output_dir}")
        
        # Read the file
        df = pd.read_csv(file, sep='\t', header=None, names=['feature_id', 'count'])
        
        # Function to process feature_id
        def process_feature_id(fid):
            # Remove leading and trailing quotes
            fid = str(fid).strip('"')
            # Check if it's a special entry
            if fid.startswith('_'):
                return pd.Series({'gene_id': fid, 'exon_id': None})
            elif ':' in fid:
                # Split into gene_id and exon_id
                gene_id, exon_id = fid.split(':')
                return pd.Series({'gene_id': gene_id, 'exon_id': exon_id.strip('"')})
            else:
                # Handle any unexpected format
                return pd.Series({'gene_id': fid, 'exon_id': None})

        # Apply the function to the feature_id column
        feature_split = df['feature_id'].apply(process_feature_id)
        df['gene_id'] = feature_split['gene_id']
        df['exon_id'] = feature_split['exon_id']

        # Rearrange columns
        df_formatted = df[['gene_id', 'exon_id', 'count']]

        # Create output filename
        output_file = os.path.join(output_dir, 
                                 os.path.basename(file).replace('.dexeq_counts', '.formatted.counts'))
        
        # Save the formatted file
        df_formatted.to_csv(output_file, sep='\t', header=False, index=False, na_rep='None')
        
        print(f"Saved formatted file to: {output_file}")
        
        # Verify file was created
        if os.path.exists(output_file):
            print(f"File successfully created with size: {os.path.getsize(output_file)} bytes")
            return output_file
        else:
            raise FileNotFoundError(f"Failed to create output file: {output_file}")

    except Exception as e:
        print(f"Error processing file {file}: {str(e)}")
        return None

# DEXSeq Files

## General Feature Format File
"DATA/gencode.v31.basic.annotation.DEXSeq.gff"


| **chr1** | **dexseq_prepare_annotation.py** | **aggregate_gene** | **11869** | **14409** | **.** | **+** | **.** | **gene_id "ENSG00000223972.5"**                                                                          |
| -------- | -------------------------------- | ------------------ | --------- | --------- | ----- | ----- | ----- | -------------------------------------------------------------------------------------------------------- |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 11869     | 12009     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000456328.2"; exonic_part_number "001"                   |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12010     | 12057     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2+ENST00000456328.2"; exonic_part_number "002" |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12058     | 12178     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000456328.2"; exonic_part_number "003"                   |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12179     | 12227     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2+ENST00000456328.2"; exonic_part_number "004" |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12613     | 12697     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2+ENST00000456328.2"; exonic_part_number "005" |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12698     | 12721     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000456328.2"; exonic_part_number "006"                   |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 12975     | 13052     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2"; exonic_part_number "007"                   |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 13221     | 13374     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2+ENST00000456328.2"; exonic_part_number "008" |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 13375     | 13452     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000456328.2"; exonic_part_number "009"                   |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 13453     | 13670     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000450305.2+ENST00000456328.2"; exonic_part_number "010" |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 13671     | 14409     | .     | +     | .     | gene_id "ENSG00000223972.5"; transcripts "ENST00000456328.2"; exonic_part_number "011"                   |
| **chr1** | **dexseq_prepare_annotation.py** | **aggregate_gene** | **14404** | **29570** | **.** | **-** | **.** | **gene_id "ENSG00000227232.5"**                                                                          |
| chr1     | dexseq_prepare_annotation.py     | exonic_part        | 14404     | 14501     | .     | -     | .     | gene_id "ENSG00000227232.5"; transcripts "ENST00000488147.1"; exonic_part_number "001"                   |

## Format Files

In [ ]:
working_dir

In [ ]:
data_dir

In [ ]:
# # Get a list of your count files
# count_files = glob.glob(os.path.join(data_dir, '*.dexeq_counts'))  # Remove 'DexSeq_counts/' from the path

# print(f"Found {len(count_files)} count files to process")
# print("\nInput files found:")
# for f in count_files:  # Print all files
#     print(f"  {f} (exists: {os.path.exists(f)})")

# # Process files in parallel with better error handling
# processed_files = []
# with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
#     future_to_file = {executor.submit(process_file, file): file for file in count_files}
#     for future in as_completed(future_to_file):
#         file = future_to_file[future]
#         try:
#             result = future.result()
#             if result:
#                 processed_files.append(result)
#                 print(f"Successfully processed: {os.path.basename(file)}")
#             else:
#                 print(f"Failed to process: {os.path.basename(file)}")
#         except Exception as exc:
#             print(f'{file} generated an exception: {exc}')

# print(f"\nSuccessfully processed {len(processed_files)} files")

# # Verify the results
# if processed_files:
#     print("\nFirst 10 lines of first processed file:")
#     with open(processed_files[0], 'r') as f:
#         for _ in range(10):
#             line = f.readline().strip()
#             print(line)
# else:
#     print("No files were successfully processed")

## Load DEXSeq count files

In [220]:
# Define paths for DEXSeq count files
dexseq_dir = os.path.join(working_dir, "DexSeq_counts")
count_files = [f for f in os.listdir(dexseq_dir) if f.endswith('.formatted.counts')]

In [ ]:
print(count_files)

## Create sample information

In [222]:
# Create sample information
sample_info = pd.DataFrame({
    'sample': [f.replace('.formatted.counts', '') for f in count_files],
    'condition': ['EDO' if f.startswith('EDO') else 'ND1' if f.startswith('ND1') else 'PW1' if f.startswith('PW1') else 'Unknown' for f in count_files],
    'replicate': [f.split('.')[0][-1] for f in count_files],
    'count_file': [os.path.join(dexseq_dir, f) for f in count_files]
})

In [ ]:
sample_info

In [224]:
# Filter for EDO and ND1 samples (excluding PW1)
edo_nd1_samples = sample_info[sample_info['condition'].isin(['EDO', 'ND1'])].copy()

In [225]:
# Ensure proper formatting for DEXSeq
edo_nd1_samples = edo_nd1_samples.reset_index(drop=True)
edo_nd1_samples['condition'] = pd.Categorical(edo_nd1_samples['condition'])
edo_nd1_samples['replicate'] = pd.Categorical(edo_nd1_samples['replicate'])

In [ ]:
edo_nd1_samples.head()

## Examine Files

In [227]:
# Get list of count files
count_files = edo_nd1_samples['count_file'].tolist()

In [ ]:
# Examine structure of files
print("Comparing file structures:")
structure_comparison = compare_file_structures(count_files)
print(structure_comparison)

In [ ]:
# Examine first file in detail
first_file = count_files[0]
df = examine_dexseq_file(first_file)

In [ ]:
df.head()

In [ ]:
# Plot overall count distribution
plt.figure(figsize=(15, 5))

plt.subplot(131)
counts = df['count'][df['count'] > 0]
plt.hist(counts, bins=50, color='skyblue', edgecolor='black', range=(0, counts.quantile(0.99)))
plt.title('Count Distribution (> 0)', fontsize=14, fontweight='bold')
plt.xlabel('Counts', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.xlim(0, counts.quantile(0.99))

plt.tight_layout()

In [ ]:
# Print outliers
outliers = df[df['count'] > counts.quantile(0.99)]
print("\nOutliers (counts above 99th percentile):")
# print(outliers[['feature_id', 'count', 'gene_id']].to_string(index=False))
display(outliers.head())

# Plot outliers distribution
plt.figure(figsize=(15, 5))

plt.subplot(131)
outlier_counts = outliers['count']
plt.hist(outlier_counts, bins=50, color='salmon', edgecolor='black')
plt.title('Outlier Count Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Counts', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.subplot(132)
plt.hist(np.log10(outlier_counts), bins=50, color='lightgreen', edgecolor='black')
plt.title('Log10 Outlier Count Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Log10(Counts)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.subplot(133)
plt.boxplot(outlier_counts)
plt.title('Outlier Count Boxplot', fontsize=14, fontweight='bold')
plt.ylabel('Counts', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.show()

# DEXSeq Analysis

In [5]:
def get_gff_features():
    """
    Extract features from GFF file
    """
    gff_features = set()
    with open("DATA/gencode.v31.basic.annotation.gff", 'r') as f:
        for line in f:
            if 'exonic_part' in line:
                fields = line.strip().split('\t')
                attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                gene_id = attrs['gene_id'].strip('"')
                exon_num = attrs['exonic_part_number'].strip('"')
                feature_id = f"{gene_id}:E{exon_num}"
                gff_features.add(feature_id)
    return gff_features

In [10]:
def create_dexseq_dataset(sample_info, processed_files, dexseq):
    """
    Create DEXSeqDataSet with proper formatting
    """
    try:
        # Prepare sample data
        sample_data = pd.DataFrame({
            'sample': sample_info['sample'],
            'condition': sample_info['condition']
        })
        
        print("\nSample data:")
        print(sample_data)
        
        # Convert to R objects
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)
        
        # Create DEXSeqDataSet
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(processed_files),
            sampleData=sample_data_r,
            design=Formula('~ sample + exon + condition:exon'),
            flattenedfile="DATA/gencode.v31.basic.annotation.gff"
        )
        
        return dxd
    
    except Exception as e:
        print(f"Error creating DEXSeq dataset: {str(e)}")
        # Add more debugging information
        print("\nChecking processed files:")
        for f in processed_files:
            print(f"\nFile: {os.path.basename(f)}")
            with open(f, 'r') as file:
                print(file.read(500))
        raise

In [ ]:
def process_count_file(file_path, gff_features):
    """
    Process DEXSeq count file to match the GFF feature format
    """
    output_dir = os.path.dirname(file_path)
    basename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, f"processed_{basename}")
    
    try:
        # Create ordered dictionary of gff features
        gff_feature_dict = {feature: True for feature in sorted(gff_features)}
        
        # Read and process file
        count_dict = {}
        with open(file_path, 'r') as f:
            for line in f:
                if not line.startswith('_'):  # Skip special entries
                    parts = line.strip().split('\t')
                    if len(parts) == 2:
                        feature_id, count = parts
                        # Remove quotes and split feature ID
                        feature_id = feature_id.strip('"')
                        if '":"' in feature_id:
                            gene_id, exon_num = feature_id.split('":"')
                            exon_num = exon_num.strip('"')
                            # Format to match GFF style
                            feature_id = f"{gene_id}:E{exon_num}"
                            
                            # Only keep features that exist in GFF
                            if feature_id in gff_feature_dict:
                                count_dict[feature_id] = int(count)
        
        # Create output with all GFF features (using 0 for missing counts)
        with open(output_path, 'w') as f:
            for feature in gff_feature_dict:
                count = count_dict.get(feature, 0)
                f.write(f"{feature}\t{count}\n")
        
        print(f"\nProcessed {basename}")
        print(f"Features written: {len(gff_feature_dict)}")
        
        # Verify file contents
        with open(output_path, 'r') as f:
            first_lines = [next(f) for _ in range(5)]
        print("First few lines:")
        for line in first_lines:
            print(line.strip())
        
        return output_path
    
    except Exception as e:
        print(f"Error processing {basename}: {str(e)}")
        return None

def verify_processed_file(file_path, gff_features):
    """
    Verify that a processed file matches GFF features exactly
    """
    try:
        file_features = set()
        with open(file_path, 'r') as f:
            for line in f:
                feature_id = line.strip().split('\t')[0]
                file_features.add(feature_id)
        
        missing_features = gff_features - file_features
        extra_features = file_features - gff_features
        
        print(f"\nVerification of {os.path.basename(file_path)}:")
        print(f"Total features in file: {len(file_features)}")
        print(f"Missing features: {len(missing_features)}")
        print(f"Extra features: {len(extra_features)}")
        
        return len(missing_features) == 0 and len(extra_features) == 0
    
    except Exception as e:
        print(f"Error verifying {file_path}: {str(e)}")
        return False

def main():
    # Set up paths
    # data_dir = "/beegfs/scratch/ric.broccoli/ric.broccoli/PW_RNA_seq_deep/DexSeq_counts"
    data_dir = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords/Create_counts/output"    
    working_dir = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords"
    os.chdir(working_dir)
    
    # Get GFF features first
    print("Loading GFF features...")
    gff_features = get_gff_features()
    print(f"Found {len(gff_features)} features in GFF")
    print("Sample GFF features:")
    for feature in sorted(list(gff_features))[:5]:
        print(feature)
    
    # Get count files
    count_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) 
                  if f.endswith('.dexeq_counts')]
    print(f"\nFound {len(count_files)} count files")
    
    # Process count files
    processed_files = []
    for file in count_files:
        processed_file = process_count_file(file, gff_features)
        if processed_file and verify_processed_file(processed_file, gff_features):
            processed_files.append(processed_file)
    
    if not processed_files:
        print("No files were processed successfully")
        return
    
    # Create sample info for EDO and ND1 samples only
    edo_nd1_samples = pd.DataFrame([
        {'sample': os.path.basename(f).replace('.dexeq_counts', '').replace('processed_', ''),
         'condition': 'EDO' if 'EDO' in f else 'ND1'}
        for f in processed_files
        if 'EDO' in f or 'ND1' in f
    ])
    
    if len(edo_nd1_samples) == 0:
        print("No EDO or ND1 samples found")
        return
    
    print("\nFinal sample information:")
    print(edo_nd1_samples)
    
    # Import DEXSeq and create dataset
    dexseq = importr('DEXSeq')
    try:
        dxd = create_dexseq_dataset(edo_nd1_samples, 
                                  [f for f in processed_files if any(s in f for s in edo_nd1_samples['sample'])],
                                  dexseq)
        print("Successfully created DEXSeq dataset!")
        return dxd
    except Exception as e:
        print(f"\nFailed to create DEXSeq dataset: {str(e)}")
        return None

if __name__ == "__main__":
    # Enable automatic conversion between pandas and R dataframes
    pandas2ri.activate()
    
    # Run the analysis
    dxd = main()

In [ ]:
# Check original count file
count_file = count_files[0]
print(f"\nOriginal count file ({os.path.basename(count_file)}):")
with open(count_file, 'r') as f:
    print(f.read(500))

# Check processed file
processed_file = processed_files[0]
print(f"\nProcessed file ({os.path.basename(processed_file)}):")
with open(processed_file, 'r') as f:
    print(f.read(500))

In [274]:
dxd

In [233]:
def create_dexseq_dataset(sample_info, formatted_files, dexseq):
    """
    Create DEXSeqDataSet with proper sample ordering and include 'sample' as a column.
    """
    try:
        # Use the sample_info DataFrame as-is since 'sample' is now a column
        # sample_data = sample_info[['sample', 'condition']]
        sample_data = sample_info[['sample', 'condition']].copy()

        # Convert columns to string type
        sample_data['sample'] = sample_data['sample'].astype(str)
        sample_data['condition'] = sample_data['condition'].astype(str)

        # Ensure formatted_files correspond to the samples
        ordered_files = sample_info['count_file'].tolist()

        print("\nSample data for DEXSeq:")
        print(sample_data)

        print("\nOrdered count files:")
        for sample, file in zip(sample_data['sample'], ordered_files):
            print(f"{sample}: {file}")

        # Convert sample_data to an R DataFrame
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)

        # Convert count files list to an R vector
        count_files_vector = ro.StrVector(ordered_files)

        # After converting to R objects
        print("R sample_data:")
        print(r.str(sample_data_r))
        print("\nR count_files_vector:")
        print(r.str(count_files_vector))

        # Also, let's check the length of these objects
        print("\nLength of sample_data_r:", r.length(sample_data_r))
        print("Length of count_files_vector:", r.length(count_files_vector))

        gff_file = "DATA/gencode.v31.basic.annotation.DEXSeq.gff"
        if not os.path.exists(gff_file):
            print(f"GFF file not found: {gff_file}")
        else:
            print(f"GFF file exists: {gff_file}")
            # Print first few lines of the GFF file
            with open(gff_file, 'r') as f:
                print(f.read(500))

        # Create DEXSeqDataSet
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=count_files_vector,
            sampleData=sample_data_r,
            # design=Formula('~ sample + exon + condition:exon'),
            design=Formula('~ condition'),
            flattenedfile=gff_file        )

        return dxd

    except Exception as e:
        print(f"Error creating DEXSeqDataSet: {str(e)}")
        raise

In [ ]:
!head DexSeq_counts/EDO_1.formatted.counts

In [ ]:
!tail DexSeq_counts/EDO_1.formatted.counts

In [ ]:
%%bash
grep -P '\texonic_part\t' DATA/gencode.v31.basic.annotation.DEXSeq.gff | \
awk 'BEGIN{FS="\t"} { 
    split($9, a, ";"); 
    for (i in a) { 
        if (a[i] ~ /gene_id/) { 
            gsub(/gene_id| |"/, "", a[i]); 
            gene_id = a[i];
        } else if (a[i] ~ /exonic_part_number/) {
            gsub(/exonic_part_number| |"/, "", a[i]); 
            exon_id = a[i];
        }
    } 
    print gene_id"\t"exon_id
}' > exonic_parts_from_gff.txt

# Display the first few lines of the output file
head exonic_parts_from_gff.txt

In [ ]:
%%bash
grep -P '\texonic_part\t' DATA/gencode.v31.basic.annotation.gff | \
awk 'BEGIN{FS="\t"} { 
    split($9, a, ";"); 
    for (i in a) { 
        if (a[i] ~ /gene_id/) { 
            gsub(/gene_id| |"/, "", a[i]); 
            gene_id = a[i];
        } else if (a[i] ~ /exonic_part_number/) {
            gsub(/exonic_part_number| |"/, "", a[i]); 
            exon_id = a[i];
        }
    } 
    print gene_id"\t"exon_id
}' > exonic_parts_from_gff.txt

# Display the first few lines of the output file
head exonic_parts_from_gff.txt

In [ ]:
!grep ENSG00000000003.14 exonic_parts_from_gff.txt

In [ ]:
!grep ENSG00000223972.5 /beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords/DexSeq_counts/EDO_1.formatted.counts

In [ ]:
!wc -l /beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords/DexSeq_counts/*.formatted.counts

In [ ]:
!head -n 10 /beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Snords/gencode.v31.basic.annotation.DEXSeq.gff

In [ ]:
sample_info

In [ ]:
count_files

In [244]:
for file in count_files:
    if not os.path.exists(file):
        print(f"File not found: {file}")

In [ ]:
with open(count_files[0], 'r') as f:
    print(f.read(100))

In [ ]:
import pandas as pd

def analyze_gff_file(gff_path):
    """
    Analyze the structure of a DEXSeq GFF file and compare with count files
    """
    print("=== GFF File Analysis ===")
    
    # Read first few lines of GFF
    print("\nFirst few lines of GFF file:")
    with open(gff_path, 'r') as f:
        header_lines = []
        feature_lines = []
        for i, line in enumerate(f):
            if i < 10:  # Store first 10 lines for analysis
                if not line.startswith('#'):
                    feature_lines.append(line.strip())
                else:
                    header_lines.append(line.strip())
    
    print("\nHeaders:")
    for line in header_lines:
        print(line)
        
    print("\nFirst few features:")
    for line in feature_lines[:3]:
        print(line)
    
    # Analyze feature structure
    features = []
    gene_exon_map = {}
    with open(gff_path, 'r') as f:
        for line in f:
            if 'exonic_part' in line:
                fields = line.strip().split('\t')
                if len(fields) >= 9:
                    attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                    gene_id = attrs.get('gene_id', '').strip('"')
                    exon_num = attrs.get('exonic_part_number', '').strip('"')
                    feature_id = f"{gene_id}:E{exon_num}"
                    features.append({
                        'feature_id': feature_id,
                        'gene_id': gene_id,
                        'exon_num': exon_num,
                        'type': fields[2],
                        'attributes': attrs
                    })
                    gene_exon_map.setdefault(gene_id, set()).add(exon_num)
    
    df = pd.DataFrame(features)
    
    print("\nFeature Statistics:")
    print(f"Total features: {len(features)}")
    print(f"Unique genes: {len(gene_exon_map)}")
    print("\nSample of feature structure:")
    print(df.head())
    
    return df, gene_exon_map

def analyze_count_file(count_path, gene_exon_map):
    """
    Analyze a count file and compare with GFF structure
    """
    print(f"\n=== Count File Analysis: {count_path} ===")
    
    missing_features = []
    with open(count_path, 'r') as f:
        count_features = []
        for i, line in enumerate(f):
            if i < 5:  # Print first few lines
                print(f"Line {i+1}: {line.strip()}")
            
            if not line.startswith('_'):
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    feature_id = parts[0]
                    # Check if this feature exists in GFF
                    gene_id = feature_id.split(':')[0] if ':' in feature_id else ''
                    exon_num = feature_id.split(':E')[1] if ':E' in feature_id else ''
                    
                    if gene_id and exon_num:
                        if gene_id not in gene_exon_map or exon_num not in gene_exon_map[gene_id]:
                            missing_features.append(feature_id)
                    
                    count_features.append(feature_id)
    
    print(f"\nTotal features in count file: {len(count_features)}")
    print(f"Missing features in GFF: {len(missing_features)}")
    if missing_features:
        print("\nSample of missing features:")
        for feature in missing_features[:5]:
            print(feature)
            
    return count_features, missing_features

# Usage example
if __name__ == "__main__":
    gff_path = "DATA/gencode.v31.basic.annotation.DEXSeq.gff"
    count_path = "DexSeq_counts/EDO_1.formatted.counts"  # Replace with your count file
    
    gff_df, gene_exon_map = analyze_gff_file(gff_path)
    count_features, missing = analyze_count_file(count_path, gene_exon_map)

In [ ]:
# Before creating DEXSeq dataset
print("\nVerifying GFF features...")
with open("DATA/gencode.v31.basic.annotation.DEXSeq.gff", 'r') as f:
    gff_features = set()
    for line in f:
        if 'exonic_part' in line:
            fields = line.strip().split('\t')
            attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
            gene_id = attrs['gene_id'].strip('"')
            exon_num = attrs['exonic_part_number'].strip('"')
            feature_id = f"{gene_id}:E{exon_num}"
            gff_features.add(feature_id)

print(f"Number of features in GFF: {len(gff_features)}")
print("\nVerifying count files...")
for pf in processed_files[:1]:  # Check first file
    with open(pf, 'r') as f:
        count_features = set(line.split('\t')[0] for line in f)
    print(f"Number of features in count file: {len(count_features)}")
    print(f"Features matching GFF: {len(count_features.intersection(gff_features))}")

In [ ]:
def process_count_file_with_mapping(file_path, ordered_features, gff_gene_mapping):
    """
    Process count file to match DEXSeq format with ordered GFF features
    """
    output_dir = os.path.dirname(file_path)
    basename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, f"fixed_{basename}")
    
    try:
        # Create a mapping of gene_id + exon_num to GFF feature_id
        feature_mapping = {}
        for feature in ordered_features:
            gene_id, exon_part = feature.split(':')
            base_gene_id = gene_id.split('.')[0]
            exon_num = exon_part[1:]  # Remove 'E' prefix
            key = (base_gene_id, exon_num)
            feature_mapping[key] = feature
        
        # Read counts into dictionary
        counts = {}
        unmapped_genes = set()
        print(f"\nProcessing {basename}")
        
        with open(file_path, 'r') as f:
            for i, line in enumerate(f):
                if not line.startswith('_'):
                    parts = line.strip().split('\t')
                    if len(parts) == 3:
                        gene_id, exon_num, count = parts
                        # Clean up IDs
                        base_gene_id = gene_id.strip('"').split('.')[0]
                        exon_num = exon_num.strip('"').zfill(3)
                        
                        # Try to find matching feature
                        key = (base_gene_id, exon_num)
                        if key in feature_mapping:
                            feature_id = feature_mapping[key]
                            counts[feature_id] = int(count)
                            if i < 5:  # Debug first few lines
                                print(f"Line {i+1}: {gene_id} -> {feature_id}")
                                print(f"Count: {count}")
                        else:
                            unmapped_genes.add(base_gene_id)
                            if i < 5:
                                print(f"Warning: No mapping for {gene_id}:E{exon_num}")

        # Report unmapped genes
        if unmapped_genes:
            print(f"\nWarning: {len(unmapped_genes)} genes not found in GFF")
            print("First few unmapped genes:", list(unmapped_genes)[:5])

        # Create output with all GFF features in the correct order
        features_with_counts = 0
        with open(output_path, 'w') as f:
            for feature_id in ordered_features:
                count = counts.get(feature_id, 0)
                if count > 0:
                    features_with_counts += 1
                f.write(f"{feature_id}\t{count}\n")
        
        print(f"\nFeatures with non-zero counts: {features_with_counts}")
        
        # Verify output format
        print("\nVerifying output file (first 5 lines):")
        with open(output_path, 'r') as f:
            for i, line in enumerate(f):
                if i < 5:
                    print(line.strip())
        
        return output_path
        
    except Exception as e:
        print(f"Error processing {basename}: {str(e)}")
        return None

def get_ordered_gff_features(gff_path):
    """
    Get list of features and create mapping from GFF file
    """
    ordered_features = []
    gene_versions = {}
    
    try:
        print("\nExamining GFF file structure:")
        with open(gff_path, 'r') as f:
            for i, line in enumerate(f):
                if 'exonic_part' in line:
                    fields = line.strip().split('\t')
                    attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                    gene_id = attrs['gene_id'].strip('"')
                    base_gene_id = gene_id.split('.')[0]
                    gene_versions[base_gene_id] = gene_id
                    
                    exon_num = attrs['exonic_part_number'].strip('"')
                    feature_id = f"{gene_id}:E{exon_num}"
                    ordered_features.append(feature_id)
                    
                    # Show example entries
                    if i < 5:
                        print(f"\nGFF entry {i+1}:")
                        print(f"Original gene_id: {attrs['gene_id']}")
                        print(f"Cleaned gene_id: {gene_id}")
                        print(f"Feature ID: {feature_id}")
    
    except Exception as e:
        print(f"Error reading GFF file: {str(e)}")
        raise
        
    return ordered_features, gene_versions

# Main processing
print("Loading GFF features...")
gff_path = "DATA/gencode.v31.basic.annotation.gff"
ordered_features, gene_versions = get_ordered_gff_features(gff_path)
print(f"\nFound {len(ordered_features)} features in GFF file")
print(f"Found {len(gene_versions)} unique genes in GFF file")
print("\nSample features:")
for feature in ordered_features[:5]:
    print(feature)

# Process all files
processed_files = []
for file in count_files:
    processed_file = process_count_file_with_mapping(file, ordered_features, gene_versions)
    if processed_file:
        processed_files.append(processed_file)

if len(processed_files) == len(edo_nd1_samples):
    print("\nAll files processed successfully")
    
    # Verify processed files
    if verify_count_files(processed_files, ordered_features):
        print("\nAll files verified successfully")
        edo_nd1_samples['count_file'] = processed_files
        
        try:
            dxd = create_dexseq_dataset(edo_nd1_samples, processed_files, dexseq)
            print("Successfully created DEXSeq dataset!")
        except Exception as e:
            print(f"\nError creating DEXSeq dataset: {str(e)}")
            print("\nDetailed error information:")
            print("Sample data:")
            print(edo_nd1_samples)
            print("\nFirst few lines of first count file:")
            with open(processed_files[0], 'r') as f:
                for i, line in enumerate(f):
                    if i < 5:
                        print(line.strip())
    else:
        print("\nSome files failed verification")
else:
    print(f"\nWarning: Only processed {len(processed_files)} out of {len(edo_nd1_samples)} files")

In [ ]:
# def process_count_file_with_mapping(file_path, ordered_features):
#     """
#     Process count file to match DEXSeq format with ordered GFF features
#     """
#     output_dir = os.path.dirname(file_path)
#     basename = os.path.basename(file_path)
#     output_path = os.path.join(output_dir, f"fixed_{basename}")
    
#     try:
#         # Create a mapping of gene_id + exon_num to GFF feature_id
#         gff_mapping = {}
#         for feature in ordered_features:
#             gene_id, exon_part = feature.split(':')
#             exon_num = exon_part[1:]  # Remove 'E' prefix
#             key = (gene_id, exon_num)
#             gff_mapping[key] = feature
        
#         # Read counts into dictionary
#         counts = {}
#         print(f"\nProcessing {basename}")
#         with open(file_path, 'r') as f:
#             for i, line in enumerate(f):
#                 if not line.startswith('_'):
#                     parts = line.strip().split('\t')
#                     if len(parts) == 3:
#                         gene_id, exon_num, count = parts
#                         # Clean up IDs
#                         gene_id = gene_id.strip('"')
#                         exon_num = exon_num.strip('"').zfill(3)
                        
#                         # Try to find matching feature
#                         key = (gene_id, exon_num)
#                         if key in gff_mapping:
#                             feature_id = gff_mapping[key]
#                             counts[feature_id] = int(count)
                        
#                         # Debug first few lines
#                         if i < 5:
#                             print(f"Line {i+1}: {gene_id}\t{exon_num}\t{count}")
#                             print(f"Mapped to feature: {gff_mapping.get(key, 'NOT FOUND')}")

#         # Create output with all GFF features in the correct order
#         with open(output_path, 'w') as f:
#             for feature_id in ordered_features:
#                 count = counts.get(feature_id, 0)
#                 f.write(f"{feature_id}\t{count}\n")
        
#         # Verify non-zero counts
#         total_counts = sum(1 for count in counts.values() if count > 0)
#         print(f"\nFeatures with non-zero counts: {total_counts}")
        
#         return output_path
        
#     except Exception as e:
#         print(f"Error processing {basename}: {str(e)}")
#         return None

# Add this function to examine the original count files
def examine_original_count_file(file_path):
    """
    Examine the structure of original count files
    """
    print(f"\nExamining {os.path.basename(file_path)}")
    try:
        with open(file_path, 'r') as f:
            print("First 5 lines:")
            for i, line in enumerate(f):
                if i < 5:
                    print(line.strip())
                    parts = line.strip().split('\t')
                    if len(parts) == 3:
                        gene_id, exon_num, count = parts
                        print(f"Parsed: gene_id='{gene_id}' exon_num='{exon_num}' count='{count}'")
    except Exception as e:
        print(f"Error examining file: {str(e)}")

# Before processing, examine original files
print("Examining original count files...")
for file in count_files[:1]:  # Check first file
    examine_original_count_file(file)

def create_dexseq_dataset(sample_info, count_files, dexseq):
    """
    Create DEXSeqDataSet with strict format checking and alignment
    """
    try:
        # Create sample data with explicit index matching count files
        sample_data = pd.DataFrame({
            'sample': sample_info['sample'],
            'condition': sample_info['condition']
        }).reset_index(drop=True)
        
        # Ensure count files match sample order
        print("\nVerifying alignment:")
        print("Sample data:")
        print(sample_data)
        print("\nCount files:")
        for cf in count_files:
            print(os.path.basename(cf))
            
        # Sort count files to match sample order
        sorted_count_files = []
        for sample in sample_data['sample']:
            matching_file = [f for f in count_files if sample in os.path.basename(f)]
            if matching_file:
                sorted_count_files.append(matching_file[0])
            else:
                raise ValueError(f"No matching count file found for sample {sample}")
        
        print("\nAligned count files:")
        for sample, cf in zip(sample_data['sample'], sorted_count_files):
            print(f"{sample}: {os.path.basename(cf)}")
        
        # Convert to R objects
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)
        
        # Create DEXSeqDataSet with verbose error checking
        print("\nCreating DEXSeqDataSet with:")
        print(f"Number of samples: {len(sample_data)}")
        print(f"Number of count files: {len(sorted_count_files)}")
        
        # Use absolute paths
        gff_path = os.path.abspath("DATA/gencode.v31.basic.annotation.DEXSeq.gff")
        sorted_count_files = [os.path.abspath(f) for f in sorted_count_files]
        
        print("\nUsing GFF file:", gff_path)
        print("First count file:", sorted_count_files[0])
        
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(sorted_count_files),
            sampleData=sample_data_r,
            design=Formula('~ sample + exon + condition:exon'),
            flattenedfile=gff_path
        )
        
        return dxd
        
    except Exception as e:
        print(f"Error creating DEXSeqDataSet: {str(e)}")
        # Additional error information
        print("\nDetailed error check:")
        print(f"Sample data shape: {sample_data.shape}")
        print("Sample data columns:", sample_data.columns.tolist())
        print("\nCount file contents verification:")
        for cf in sorted_count_files[:1]:  # Check first file
            print(f"\nChecking {os.path.basename(cf)}")
            try:
                with open(cf, 'r') as f:
                    first_lines = [next(f) for _ in range(5)]
                print("First 5 lines:")
                for line in first_lines:
                    print(line.strip())
            except Exception as e:
                print(f"Error reading file: {e}")
        raise

def get_ordered_gff_features(gff_path):
    """
    Get list of features from GFF file in correct order
    """
    ordered_features = []
    try:
        with open(gff_path, 'r') as f:
            for line in f:
                if 'exonic_part' in line:
                    fields = line.strip().split('\t')
                    attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                    gene_id = attrs['gene_id'].strip('"')
                    exon_num = attrs['exonic_part_number'].strip('"')
                    feature_id = f"{gene_id}:E{exon_num}"
                    ordered_features.append(feature_id)
    except Exception as e:
        print(f"Error reading GFF file: {str(e)}")
        raise
        
    return ordered_features

def verify_count_files(count_files, ordered_features):
    """
    Verify all count files have correct format and features
    """
    print("\nVerifying count files...")
    all_valid = True
    
    for cf in count_files:
        print(f"\nChecking {os.path.basename(cf)}")
        try:
            with open(cf, 'r') as f:
                file_features = [line.split('\t')[0] for line in f]
            
            if file_features == ordered_features:
                print("✓ Features match GFF order")
            else:
                print("✗ Features do not match GFF order")
                all_valid = False
                
                # Show first mismatch
                for i, (expected, actual) in enumerate(zip(ordered_features, file_features)):
                    if expected != actual:
                        print(f"First mismatch at position {i}:")
                        print(f"  Expected: {expected}")
                        print(f"  Found: {actual}")
                        break
        except Exception as e:
            print(f"Error verifying file: {str(e)}")
            all_valid = False
    
    return all_valid

# Main processing
print("Loading GFF features...")
gff_path = "DATA/gencode.v31.basic.annotation.DEXSeq.gff"
ordered_features = get_ordered_gff_features(gff_path)
print(f"Found {len(ordered_features)} features in GFF file")
print("Sample features:")
for feature in ordered_features[:5]:
    print(feature)

# Process all files
# processed_files = []
# for file in count_files:
#     processed_file = process_count_file_with_mapping(file, ordered_features)
#     if processed_file:
#         processed_files.append(processed_file)

if len(processed_files) == len(edo_nd1_samples):
    print("\nAll files processed successfully")
    
    # Verify processed files
    if verify_count_files(processed_files, ordered_features):
        print("\nAll files verified successfully")
        edo_nd1_samples['count_file'] = processed_files
        
        try:
            dxd = create_dexseq_dataset(edo_nd1_samples, processed_files, dexseq)
            print("Successfully created DEXSeq dataset!")
        except Exception as e:
            print(f"\nError creating DEXSeq dataset: {str(e)}")
            print("\nChecking file formats:")
            for pf in processed_files:
                print(f"\nFile: {os.path.basename(pf)}")
                with open(pf, 'r') as f:
                    for i, line in enumerate(f):
                        if i < 5:  # Print first 5 lines
                            print(line.strip())
    else:
        print("\nSome files failed verification")
else:
    print(f"\nWarning: Only processed {len(processed_files)} out of {len(edo_nd1_samples)} files")

In [ ]:
import pandas as pd
import os
from pathlib import Path

def parse_gff_line(line):
    """Parse a GFF line and return structured data"""
    fields = line.strip().split('\t')
    if len(fields) < 9:
        return None
        
    attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
    return {
        'chromosome': fields[0],
        'source': fields[1],
        'feature': fields[2],
        'start': fields[3],
        'end': fields[4],
        'strand': fields[6],
        'attributes': attrs,
        'raw_line': line.strip()
    }

def analyze_gff_format(gff_path):
    """Detailed analysis of GFF file format"""
    print(f"\nAnalyzing GFF file: {gff_path}")
    
    features = []
    attributes_seen = set()
    feature_types = set()
    
    with open(gff_path, 'r') as f:
        for i, line in enumerate(f):
            if line.startswith('#') or not line.strip():
                continue
                
            parsed = parse_gff_line(line)
            if parsed:
                feature_types.add(parsed['feature'])
                attributes_seen.update(parsed['attributes'].keys())
                
                if parsed['feature'] == 'exonic_part':
                    gene_id = parsed['attributes'].get('gene_id', '').strip('"')
                    exon_num = parsed['attributes'].get('exonic_part_number', '').strip('"')
                    if gene_id and exon_num:
                        features.append({
                            'feature_id': f"{gene_id}:E{exon_num}",
                            'gene_id': gene_id,
                            'exon_num': exon_num,
                            'line_num': i + 1
                        })
    
    print("\nGFF Analysis:")
    print(f"Total features: {len(features)}")
    print(f"Feature types: {feature_types}")
    print(f"Attributes found: {attributes_seen}")
    print("\nFirst few features:")
    for f in features[:5]:
        print(f"Line {f['line_num']}: {f['feature_id']}")
    
    return features

def analyze_count_file(count_path):
    """Detailed analysis of count file format"""
    print(f"\nAnalyzing count file: {count_path}")
    
    features = []
    with open(count_path, 'r') as f:
        for i, line in enumerate(f):
            if line.startswith('_') or not line.strip():
                continue
                
            parts = line.strip().split('\t')
            if len(parts) == 2:
                feature_id, count = parts
                features.append({
                    'feature_id': feature_id,
                    'count': count,
                    'line_num': i + 1
                })
    
    print(f"\nCount file analysis:")
    print(f"Total features: {len(features)}")
    print("\nFirst few features:")
    for f in features[:5]:
        print(f"Line {f['line_num']}: {f['feature_id']} -> {f['count']}")
    
    return features

def compare_formats(gff_features, count_features):
    """Compare GFF and count file features"""
    print("\nComparing formats:")
    
    gff_ids = set(f['feature_id'] for f in gff_features)
    count_ids = set(f['feature_id'] for f in count_features)
    
    missing_in_counts = gff_ids - count_ids
    extra_in_counts = count_ids - gff_ids
    
    print(f"\nFeatures in GFF: {len(gff_ids)}")
    print(f"Features in counts: {len(count_ids)}")
    print(f"Missing in counts: {len(missing_in_counts)}")
    print(f"Extra in counts: {len(extra_in_counts)}")
    
    if missing_in_counts:
        print("\nSample missing features:")
        for feat in list(missing_in_counts)[:5]:
            print(f"  {feat}")
    
    if extra_in_counts:
        print("\nSample extra features:")
        for feat in list(extra_in_counts)[:5]:
            print(f"  {feat}")
    
    # Check feature ID format
    print("\nFeature ID format analysis:")
    gff_sample = list(gff_ids)[0]
    count_sample = list(count_ids)[0]
    print(f"GFF format:   {gff_sample}")
    print(f"Count format: {count_sample}")
    
    return len(missing_in_counts) == 0 and len(extra_in_counts) == 0

def fix_count_file(count_path, gff_features, output_dir=None):
    """Fix count file based on GFF features"""
    if output_dir is None:
        output_dir = Path(count_path).parent / "fixed"
    os.makedirs(output_dir, exist_ok=True)
    
    output_path = os.path.join(output_dir, f"fixed_{Path(count_path).name}")
    
    # Create ordered dict of GFF features
    gff_ordered = {f['feature_id']: i for i, f in enumerate(gff_features)}
    
    # Read current counts
    counts = {}
    with open(count_path, 'r') as f:
        for line in f:
            if not line.startswith('_'):
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    feature_id, count = parts
                    counts[feature_id] = int(count)
    
    # Write fixed file with exact GFF order
    with open(output_path, 'w') as f:
        for feature in gff_features:
            feature_id = feature['feature_id']
            count = counts.get(feature_id, 0)
            f.write(f"{feature_id}\t{count}\n")
    
    return output_path

if __name__ == "__main__":
    # File paths
    gff_path = "DATA/gencode.v31.basic.annotation.DEXSeq.gff"
    count_path = "DexSeq_counts/fixed_EDO_1.formatted.counts"  # Update with your count file
    
    # Analyze files
    gff_features = analyze_gff_format(gff_path)
    count_features = analyze_count_file(count_path)
    
    # Compare formats
    formats_match = compare_formats(gff_features, count_features)
    
    if not formats_match:
        print("\nFixing count file...")
        fixed_path = fix_count_file(count_path, gff_features)
        print(f"Fixed file written to: {fixed_path}")
        
        # Verify fix
        fixed_features = analyze_count_file(fixed_path)
        fixed_match = compare_formats(gff_features, fixed_features)
        if fixed_match:
            print("\n✓ File successfully fixed")
        else:
            print("\n✗ Issues remain after fixing")

In [ ]:
def process_count_file_with_mapping(file_path, gff_features):
    output_dir = os.path.dirname(file_path)
    basename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, f"fixed_{basename}")
    
    # Create ordered list of features from GFF
    ordered_features = []
    with open("DATA/gencode.v31.basic.annotation.DEXSeq.gff", 'r') as f:
        for line in f:
            if 'exonic_part' in line:
                fields = line.strip().split('\t')
                attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                gene_id = attrs['gene_id'].strip('"')
                exon_num = attrs['exonic_part_number'].strip('"')
                feature_id = f"{gene_id}:E{exon_num}"
                ordered_features.append(feature_id)
    
    # Read counts into dictionary
    counts = {}
    with open(file_path, 'r') as f:
        for line in f:
            if not line.startswith('_'):
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    gene_id, exon_num, count = parts
                    gene_id = gene_id.strip('"')
                    exon_num = exon_num.strip('"').zfill(3)
                    feature_id = f"{gene_id}:E{exon_num}"
                    counts[feature_id] = int(count)
    
    # Create output with all GFF features in order
    with open(output_path, 'w') as f:
        for feature_id in ordered_features:
            count = counts.get(feature_id, 0)  # Use 0 for missing features
            f.write(f"{feature_id}\t{count}\n")
    
    return output_path

def create_dexseq_dataset(sample_info, count_files, dexseq):
    """
    Create DEXSeqDataSet with strict format checking
    """
    try:
        # Create sample data
        sample_data = pd.DataFrame({
            'sample': sample_info['sample'],
            'condition': sample_info['condition']
        })
        
        print("\nSample data:")
        print(sample_data)
        
        # Convert to R objects
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)
        
        # Create DEXSeqDataSet
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(count_files),
            sampleData=sample_data_r,
            # design=Formula('~ condition'),  # Simplified design
            design=Formula('~ sample + exon + condition:exon'),
            flattenedfile="DATA/gencode.v31.basic.annotation.DEXSeq.gff"
        )
        
        return dxd
        
    except Exception as e:
        print(f"Error creating DEXSeqDataSet: {str(e)}")
        raise

# First, load and parse GFF features
print("Loading GFF features...")
gff_features = set()
with open("DATA/gencode.v31.basic.annotation.DEXSeq.gff", 'r') as f:
    for line in f:
        if 'exonic_part' in line:
            fields = line.strip().split('\t')
            attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
            gene_id = attrs['gene_id'].strip('"')
            exon_num = attrs['exonic_part_number'].strip('"')
            feature_id = f"{gene_id}:E{exon_num}"
            gff_features.add(feature_id)

print(f"Found {len(gff_features)} features in GFF file")
print("Sample features:")
for feature in list(gff_features)[:5]:
    print(feature)

# Process all files
processed_files = []
for file in count_files:
    processed_file = process_count_file_with_mapping(file, gff_features)
    if processed_file:
        processed_files.append(processed_file)

if len(processed_files) == len(edo_nd1_samples):
    print("\nAll files processed successfully")
    edo_nd1_samples['count_file'] = processed_files
    
    try:
        dxd = create_dexseq_dataset(edo_nd1_samples, processed_files, dexseq)
        print("Successfully created DEXSeq dataset!")
    except Exception as e:
        print(f"\nError creating DEXSeq dataset: {str(e)}")
        # Print detailed debugging information
        print("\nChecking file formats:")
        for pf in processed_files:
            print(f"\nFile: {os.path.basename(pf)}")
            with open(pf, 'r') as f:
                for i, line in enumerate(f):
                    if i < 5:  # Print first 5 lines
                        print(line.strip())
else:
    print(f"\nWarning: Only processed {len(processed_files)} out of {len(edo_nd1_samples)} files")

In [ ]:
def process_count_file_with_mapping(file_path, ordered_features):
    """
    Process count file to match DEXSeq format with ordered GFF features.
    DEXSeq expects the format: gene_id exon_number count
    """
    output_dir = os.path.dirname(file_path)
    basename = os.path.basename(file_path)
    output_path = os.path.join(output_dir, f"fixed_{basename}")
    
    try:
        # Read counts into dictionary
        counts = {}
        with open(file_path, 'r') as f:
            for line in f:
                if not line.startswith('_'):
                    parts = line.strip().split('\t')
                    if len(parts) == 2:  # Current format: GENE:EXON count
                        feature_id, count = parts
                        # Split feature ID into gene and exon
                        gene_id, exon_part = feature_id.split(':')
                        exon_num = exon_part[1:]  # Remove 'E' prefix
                        counts[feature_id] = int(count)

        # Create output with DEXSeq expected format
        with open(output_path, 'w') as f:
            for feature_id in ordered_features:
                # Split feature ID into components
                gene_id, exon_part = feature_id.split(':')
                exon_num = exon_part[1:]  # Remove 'E' prefix
                count = counts.get(feature_id, 0)
                # Write in DEXSeq format: gene_id exon_number count
                f.write(f'{gene_id}\t{exon_num}\t{count}\n')
        
        # Verify the output
        print(f"\nProcessing {basename}")
        feature_count = len(ordered_features)
        print(f"Written {feature_count} features")
        print("\nSample of processed data:")
        with open(output_path, 'r') as f:
            for i, line in enumerate(f):
                if i < 5:
                    print(line.strip())
        
        return output_path
        
    except Exception as e:
        print(f"Error processing {basename}: {str(e)}")
        return None

def get_ordered_gff_features(gff_path):
    """
    Get list of features from GFF file in correct order
    """
    ordered_features = []
    try:
        with open(gff_path, 'r') as f:
            for line in f:
                if 'exonic_part' in line:
                    fields = line.strip().split('\t')
                    attrs = dict(item.strip().split(' ', 1) for item in fields[8].strip().split(';'))
                    gene_id = attrs['gene_id'].strip('"')
                    exon_num = attrs['exonic_part_number'].strip('"')
                    feature_id = f"{gene_id}:E{exon_num}"
                    ordered_features.append(feature_id)
    except Exception as e:
        print(f"Error reading GFF file: {str(e)}")
        raise
        
    return ordered_features

def verify_count_files(count_files):
    """
    Verify all count files have correct DEXSeq format
    """
    print("\nVerifying count files...")
    all_valid = True
    
    for cf in count_files:
        print(f"\nChecking {os.path.basename(cf)}")
        try:
            with open(cf, 'r') as f:
                for i, line in enumerate(f):
                    if i < 5:  # Print first few lines for verification
                        print(line.strip())
                    # Check format: should be gene_id exon_number count
                    parts = line.strip().split('\t')
                    if len(parts) != 3:
                        print(f"✗ Invalid format in line {i+1}: expected 3 columns, got {len(parts)}")
                        all_valid = False
                        break
                    gene_id, exon_num, count = parts
                    try:
                        int(exon_num)
                        int(count)
                    except ValueError:
                        print(f"✗ Invalid numeric format in line {i+1}")
                        all_valid = False
                        break
        except Exception as e:
            print(f"Error verifying file: {str(e)}")
            all_valid = False
    
    return all_valid

# Main processing
print("Loading GFF features...")
gff_path = "DATA/gencode.v31.basic.annotation.DEXSeq.gff"
ordered_features = get_ordered_gff_features(gff_path)
print(f"Found {len(ordered_features)} features in GFF file")
print("Sample features:")
for feature in ordered_features[:5]:
    print(feature)

# Process all files
processed_files = []
for file in count_files:
    processed_file = process_count_file_with_mapping(file, ordered_features)
    if processed_file:
        processed_files.append(processed_file)

if len(processed_files) == len(edo_nd1_samples):
    print("\nAll files processed successfully")
    
    # Verify processed files
    if verify_count_files(processed_files):
        print("\nAll files verified successfully")
        edo_nd1_samples['count_file'] = processed_files
        
        try:
            dxd = create_dexseq_dataset(edo_nd1_samples, processed_files, dexseq)
            print("Successfully created DEXSeq dataset!")
        except Exception as e:
            print(f"\nError creating DEXSeq dataset: {str(e)}")
            print("\nChecking file formats:")
            for pf in processed_files:
                print(f"\nFile: {os.path.basename(pf)}")
                with open(pf, 'r') as f:
                    for i, line in enumerate(f):
                        if i < 5:  # Print first 5 lines
                            print(line.strip())
    else:
        print("\nSome files failed verification")
else:
    print(f"\nWarning: Only processed {len(processed_files)} out of {len(edo_nd1_samples)} files")

In [ ]:
def create_dexseq_dataset(sample_info, count_files, dexseq):
    """
    Create DEXSeqDataSet with strict format checking and alignment
    """
    try:
        # Create sample data with explicit index matching count files
        sample_data = pd.DataFrame({
            'sample': sample_info['sample'],
            'condition': sample_info['condition']
        }).reset_index(drop=True)
        
        # Ensure count files match sample order
        print("\nVerifying alignment:")
        print("Sample data:")
        print(sample_data)
        print("\nCount files:")
        for cf in count_files:
            print(os.path.basename(cf))
            
        # Sort count files to match sample order
        sorted_count_files = []
        for sample in sample_data['sample']:
            matching_file = [f for f in count_files if sample in os.path.basename(f)]
            if matching_file:
                sorted_count_files.append(matching_file[0])
            else:
                raise ValueError(f"No matching count file found for sample {sample}")
        
        print("\nAligned count files:")
        for sample, cf in zip(sample_data['sample'], sorted_count_files):
            print(f"{sample}: {os.path.basename(cf)}")
        
        # Convert to R objects
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)
        
        # Create DEXSeqDataSet with verbose error checking
        print("\nCreating DEXSeqDataSet with:")
        print(f"Number of samples: {len(sample_data)}")
        print(f"Number of count files: {len(sorted_count_files)}")
        
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(sorted_count_files),
            sampleData=sample_data_r,
            design=Formula('~ sample + exon + condition:exon'),
            flattenedfile="DATA/gencode.v31.basic.annotation.DEXSeq.gff"
        )
        
        return dxd
        
    except Exception as e:
        print(f"Error creating DEXSeqDataSet: {str(e)}")
        # Additional error information
        print("\nDetailed error check:")
        print(f"Sample data shape: {sample_data.shape}")
        print("Sample data columns:", sample_data.columns.tolist())
        print("\nCount file contents verification:")
        for cf in sorted_count_files:
            print(f"\nChecking {os.path.basename(cf)}")
            try:
                with open(cf, 'r') as f:
                    first_lines = [next(f) for _ in range(5)]
                print("First 5 lines:")
                for line in first_lines:
                    print(line.strip())
            except Exception as e:
                print(f"Error reading file: {e}")
        raise

def verify_sample_data(sample_info, count_files):
    """
    Verify sample data matches count files exactly
    """
    print("\nVerifying sample data and count files alignment:")
    
    # Check sample counts
    print(f"Number of samples in sample_info: {len(sample_info)}")
    print(f"Number of count files: {len(count_files)}")
    
    # Create mapping of samples to files
    sample_file_map = {}
    for cf in count_files:
        basename = os.path.basename(cf)
        for sample in sample_info['sample']:
            if sample in basename:
                sample_file_map[sample] = cf
                break
    
    # Check for missing mappings
    missing_samples = set(sample_info['sample']) - set(sample_file_map.keys())
    extra_files = [cf for cf in count_files if not any(s in os.path.basename(cf) for s in sample_info['sample'])]
    
    if missing_samples:
        print("\nWARNING: Samples without matching files:")
        for sample in missing_samples:
            print(f"  - {sample}")
    
    if extra_files:
        print("\nWARNING: Count files without matching samples:")
        for ef in extra_files:
            print(f"  - {os.path.basename(ef)}")
    
    # Print mapping
    print("\nSample to file mapping:")
    for sample in sample_info['sample']:
        matched_file = sample_file_map.get(sample, "NO MATCH")
        if isinstance(matched_file, str):
            matched_file = os.path.basename(matched_file)
        print(f"  {sample} -> {matched_file}")
    
    return len(missing_samples) == 0 and len(extra_files) == 0

# Update main processing code
if len(processed_files) == len(edo_nd1_samples):
    print("\nAll files processed successfully")
    
    # Verify sample data alignment
    if verify_sample_data(edo_nd1_samples, processed_files):
        print("\nSample data and count files aligned correctly")
        edo_nd1_samples['count_file'] = processed_files
        
        try:
            # Create the DEXSeq dataset
            dxd = create_dexseq_dataset(edo_nd1_samples, processed_files, dexseq)
            print("Successfully created DEXSeq dataset!")
        except Exception as e:
            print(f"\nError creating DEXSeq dataset: {str(e)}")
            # Print sample data for debugging
            print("\nSample data:")
            print(edo_nd1_samples)
            print("\nProcessed files:")
            for pf in processed_files:
                print(os.path.basename(pf))
    else:
        print("\nSample data and count files misaligned")
else:
    print(f"\nWarning: Only processed {len(processed_files)} out of {len(edo_nd1_samples)} files")

In [ ]:
dxd

In [ ]:
# Check the first few lines of an original count file
with open(count_files[0], 'r') as f:
    print("First 10 lines of original count file:")
    print(f.read(500))  # First 500 characters

# Check the data types of the values
df = pd.read_csv(count_files[0], sep='\s+', header=None)
print("\nData types:")
print(df.dtypes)
print("\nSample of unique values in first column:")
print(df[0].unique()[:10])

In [ ]:
def check_count_file_format(file_path):
    """
    Check the format of a count file in detail
    """
    print(f"\nChecking file: {file_path}")
    
    # Read first few lines
    with open(file_path, 'r') as f:
        lines = f.readlines()[:5]
    
    print("\nFirst 5 lines:")
    for i, line in enumerate(lines, 1):
        parts = line.strip().split('\t')
        print(f"\nLine {i}:")
        print(f"Number of columns: {len(parts)}")
        print(f"Content: {parts}")
        
        # Check format of feature ID
        feature_id = parts[0]
        if ':' in feature_id:
            gene_id, exon_num = feature_id.split(':')
            print(f"Gene ID: {gene_id}")
            print(f"Exon number: {exon_num}")
        else:
            print(f"Feature ID format: {feature_id}")

# Check the first processed file
check_count_file_format(processed_files[0])

In [ ]:
print(edo_nd1_samples)
print(count_files)
print("\n")
print(dexseq)


print(len(edo_nd1_samples))
print(len(processed_files))


In [ ]:
dxd = create_dexseq_dataset(edo_nd1_samples, count_files, dexseq)

In [47]:

def format_count_file(input_file, output_dir):
    """
    Format count file for DEXSeq with proper space handling
    """
    output_file = os.path.join(output_dir, os.path.basename(input_file).replace('.dexeq_counts', '.formatted.counts'))
    
    try:
        print(f"\nProcessing file: {os.path.basename(input_file)}")
        
        # Read file using tab separator
        df = pd.read_csv(input_file, sep='\t', header=None, 
                         names=['feature_id', 'count'],
                         quoting=3)  # Turn off quoting
        
        print(f"Read {len(df)} lines from file")
        print("\nFirst few lines of input:")
        print(df.head())
        
        # Process feature IDs
        processed_data = []
        for _, row in df.iterrows():
            feature_id = row['feature_id'].strip('"')  # Remove outer quotes
            count = row['count']
            
            # Handle special entries
            if feature_id.startswith('_'):
                processed_data.append([feature_id, '0', str(int(count))])
            else:
                try:
                    # Split feature ID into components
                    gene_id, exon_num = feature_id.split('":"')
                    exon_num = exon_num.strip('"')
                    
                    # Format line for DEXSeq
                    processed_data.append([gene_id, exon_num, str(int(count))])
                except Exception as e:
                    print(f"Error processing line {feature_id}: {str(e)}")
                    continue
        
        if not processed_data:
            raise ValueError("No valid data was processed")
            
        # Create and save formatted DataFrame
        formatted_df = pd.DataFrame(processed_data, 
                                    columns=['gene_id', 'exon_id', 'count'])
        
        # Save without index and headers, tab-separated
        formatted_df.to_csv(output_file, sep='\t', index=False, header=False)
        
        print(f"\nProcessed file: {os.path.basename(output_file)}")
        print("First few lines of output:")
        print(formatted_df.head().to_string(index=False))
        
        return output_file
        
    except Exception as e:
        print(f"Error formatting file {input_file}: {str(e)}")
        return None

def prepare_sample_info(sample_info):
    """
    Prepare sample information DataFrame
    """
    try:
        # Filter for EDO and ND1 samples
        samples = sample_info[sample_info['condition'].isin(['EDO', 'ND1'])].copy()
        
        # Reset index
        samples = samples.reset_index(drop=True)
        
        # Convert data types
        samples['sample'] = samples['sample'].astype(str)
        samples['condition'] = pd.Categorical(samples['condition'])
        samples['replicate'] = samples['replicate'].astype(str)
        
        # Set sample as index
        samples.set_index('sample', inplace=True)
        
        return samples
        
    except Exception as e:
        print(f"Error preparing sample info: {str(e)}")
        return None

def create_dexseq_dataset(sample_info, formatted_files, dexseq):
    """
    Create DEXSeqDataSet with proper sample ordering and include 'sample' as a column.
    """
    try:
        # Reset index to have 'sample' as a column
        sample_data = sample_info.reset_index()

        # Ensure 'sample' and 'condition' are present
        sample_data = sample_data[['sample', 'condition']]

        # Convert 'sample' and 'condition' to string (if not already)
        sample_data['sample'] = sample_data['sample'].astype(str)
        sample_data['condition'] = sample_data['condition'].astype(str)

        # Create a mapping of sample names to their corresponding count files
        sample_to_file = {}
        for file_path in formatted_files:
            sample_name = os.path.splitext(os.path.basename(file_path))[0].replace('.formatted', '')
            sample_to_file[sample_name] = file_path

        # Order the count files to match the sample_data order
        ordered_files = []
        for sample in sample_data['sample']:
            if sample not in sample_to_file:
                raise ValueError(f"Sample {sample} not found in count files")
            ordered_files.append(sample_to_file[sample])

        print("\nSample data for DEXSeq:")
        print(sample_data)

        print("\nOrdered count files:")
        for sample, file in zip(sample_data['sample'], ordered_files):
            print(f"{sample}: {os.path.basename(file)}")

        # Convert to R
        with localconverter(ro.default_converter + pandas2ri.converter):
            sample_data_r = ro.conversion.py2rpy(sample_data)

        # Create count files vector with correct ordering
        count_files_vector = ro.StrVector(ordered_files)

        # Create DEXSeqDataSet
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=count_files_vector,
            sampleData=sample_data_r,
            design=Formula('~ sample + exon + condition:exon'),
            flattenedfile="/beegfs/scratch/ric.broccoli/ric.broccoli/PW_RNA_seq_deep/gencode.v31.basic.annotation.DEXSeq.gff"
        )

        return dxd

    except Exception as e:
        print(f"Error creating DEXSeqDataSet: {str(e)}")
        raise

In [ ]:
# Create output directory
temp_dir = "temp_dexseq_counts"
os.makedirs(temp_dir, exist_ok=True)

# Format count files
print("\nFormatting count files...")
formatted_files = []
for _, row in edo_nd1_samples.iterrows():
    formatted_file = format_count_file(row['count_file'], temp_dir)
    if formatted_file:
        formatted_files.append(formatted_file)

if not formatted_files:
    raise ValueError("No files were formatted successfully")

In [ ]:
# Extract sample names from formatted files
sample_names = [os.path.splitext(os.path.basename(file))[0] for file in formatted_files]

# Prepare sample information
print("\nPreparing sample information...")
# Reset index to include 'sample' as a column
sample_info = edo_nd1_samples.reset_index()

# Update 'count_file' paths to the formatted files
sample_info['count_file'] = formatted_files

# Update 'sample' column with the correct names
sample_info['sample'] = sample_names

# Ensure proper data types
sample_info['sample'] = sample_info['sample'].astype(str)
sample_info['condition'] = sample_info['condition'].astype(str)
sample_info['replicate'] = sample_info['replicate'].astype(str)

# Now, sample_info should have 'sample' as a column with the correct sample names
print("\nUpdated sample_info DataFrame:")
print(sample_info)

In [ ]:
# Create DEXSeqDataSet
print("\nCreating DEXSeqDataSet...")
dexseq = importr('DEXSeq')
dxd = create_dexseq_dataset(sample_info, formatted_files, dexseq)

In [ ]:
sample_info

In [ ]:
formatted_files

In [ ]:
# Import R packages and create DEXSeqDataSet
print("\nCreating DEXSeqDataSet...")
dexseq = importr('DEXSeq')
dxd = create_dexseq_dataset(sample_info, formatted_files, dexseq)

In [ ]:
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)


In [ ]:
# Create DEXSeqDataSet with the filtered samples
dxd = dexseq.DEXSeqDataSetFromHTSeq(
    countfiles=ro.StrVector(edo_nd1_samples['count_file'].tolist()),
    sampleData=edo_nd1_samples,
    design=r('~ sample + exon + condition:exon'),
    flattenedfile=os.path.join(working_dir, "gencode.v31.basic.annotation.gff")
)

In [101]:
import os
import pandas as pd
import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, r
import rpy2.robjects as ro
import matplotlib.pyplot as plt
import seaborn as sns

# Enable automatic conversion between pandas and R dataframes
pandas2ri.activate()

def format_count_file_for_dexseq(input_file, output_file):
    """
    Reformat count file to match DEXSeq expectations:
    From: "ENSG00000000003.14":"001"    107
    To:   ENSG00000000003.14    001    107
    """
    try:
        # Read the original file
        df = pd.read_csv(input_file, sep='\t', header=None, 
                        names=['feature_id', 'count'])
        
        # Split the feature_id into gene_id and exon_number
        df['gene_id'] = df['feature_id'].apply(lambda x: x.split(':')[0].strip('"'))
        df['exon_number'] = df['feature_id'].apply(lambda x: x.split(':')[1].strip('"') if ':' in x else None)
        
        # Write in DEXSeq expected format
        with open(output_file, 'w') as f:
            for _, row in df.iterrows():
                if row['exon_number'] is not None:
                    f.write(f"{row['gene_id']}\t{row['exon_number']}\t{row['count']}\n")
                else:
                    # Handle special cases like _ambiguous, _empty, etc.
                    f.write(f"{row['feature_id']}\t0\t{row['count']}\n")
                    
        return True
    except Exception as e:
        print(f"Error formatting file {input_file}: {str(e)}")
        return False

def prepare_sample_info(sample_info_df):
    """
    Prepare sample information and reformat count files.
    """
    # Filter for EDO and ND1 samples
    edo_nd1_samples = sample_info_df[sample_info_df['condition'].isin(['EDO', 'ND1'])].copy()
    edo_nd1_samples = edo_nd1_samples.reset_index(drop=True)
    
    # Create reformatted count files
    reformatted_files = []
    for idx, row in edo_nd1_samples.iterrows():
        original_file = row['count_file']
        reformatted_file = original_file + '.reformatted'
        if format_count_file_for_dexseq(original_file, reformatted_file):
            reformatted_files.append(reformatted_file)
        else:
            raise Exception(f"Failed to format count file: {original_file}")
    
    # Update count_file paths
    edo_nd1_samples['count_file'] = reformatted_files
    
    # Ensure proper categorical variables
    edo_nd1_samples['condition'] = pd.Categorical(edo_nd1_samples['condition'])
    edo_nd1_samples['replicate'] = pd.Categorical(edo_nd1_samples['replicate'])
    
    return edo_nd1_samples

def run_dexseq_analysis(sample_info_df, working_dir, padj_threshold=0.05, log2fc_threshold=1.0):
    """
    Run DEXSeq analysis with reformatted files.
    """
    print("Sample groups in data:")
    print(sample_info_df['condition'].value_counts())
    
    # Prepare samples and reformat files
    print("\nPreparing count files...")
    edo_nd1_samples = prepare_sample_info(sample_info_df)
    print("\nFiltered samples:")
    print(edo_nd1_samples)
    
    try:
        # Import R packages
        dexseq = importr('DEXSeq')
        deseq2 = importr('DESeq2')
        
        # Create DEXSeqDataSet
        print("\nCreating DEXSeqDataSet...")
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(edo_nd1_samples['count_file'].tolist()),
            sampleData=edo_nd1_samples,
            design=r('~ sample + exon + condition:exon'),
            flattenedfile=os.path.join(working_dir, "gencode.v31.basic.annotation.gff")
        )
        
        # Continue with analysis as before...
        print("Estimating size factors...")
        dxd = dexseq.estimateSizeFactors(dxd)
        
        print("Estimating dispersions...")
        r.pdf("dispersion_estimates.pdf")
        dxd = dexseq.estimateDispersions(dxd, quiet=False)
        r('dev.off()')
        
        print("Testing for differential exon usage...")
        dxd = dexseq.testForDEU(dxd)
        dxd = dexseq.estimateExonFoldChanges(dxd, fitExpToVar="condition")
        
        # Get results
        print("Processing results...")
        res = dexseq.DEXSeqResults(dxd)
        res_df = pandas2ri.rpy2py_dataframe(res)
        
        # Process results
        return process_results(res_df, padj_threshold, log2fc_threshold)
        
    except Exception as e:
        print(f"Error in DEXSeq analysis: {str(e)}")
        raise
    finally:
        # Clean up reformatted files
        for file in edo_nd1_samples['count_file']:
            if os.path.exists(file):
                os.remove(file)

def process_results(res_df, padj_threshold, log2fc_threshold):
    """
    Process DEXSeq results with corrected statistics.
    """
    # Add detailed statistics
    results_with_stats = res_df.copy()
    results_with_stats['log2FoldChange'] = np.log2(res_df['exonBaseMean'].replace(0, np.nan))
    
    # Handle potential infinity values
    results_with_stats['log2FoldChange'] = results_with_stats['log2FoldChange'].replace([np.inf, -np.inf], np.nan)
    
    # Calculate percentage usage
    results_with_stats['percentageUsage'] = (res_df['countData'] / 
                                           res_df['exonBaseMean'].replace(0, np.nan) * 100)

    # Filter significant results
    significant_results = results_with_stats[
        (results_with_stats['padj'] < padj_threshold) & 
        (abs(results_with_stats['log2FoldChange']) > log2fc_threshold)
    ].copy()

    # Generate summary statistics
    summary_stats = create_summary_statistics(results_with_stats, significant_results)
    
    # Create visualizations
    create_visualization_plots(results_with_stats, significant_results, 
                             padj_threshold, log2fc_threshold)

    # Save results
    save_results(results_with_stats, significant_results, summary_stats)

    return results_with_stats, significant_results, summary_stats

def create_summary_statistics(results_df, significant_df):
    """
    Create summary statistics with proper handling of NaN values.
    """
    return pd.DataFrame({
        'Metric': [
            'Total Exons Tested',
            'Significant Exons (padj < 0.05)',
            'Mean Log2 Fold Change (significant)',
            'Median adjusted p-value (significant)',
            'Number of Upregulated Exons',
            'Number of Downregulated Exons'
        ],
        'Value': [
            len(results_df),
            len(significant_df),
            significant_df['log2FoldChange'].mean(),
            significant_df['padj'].median(),
            sum(significant_df['log2FoldChange'] > 0),
            sum(significant_df['log2FoldChange'] < 0)
        ]
    })

def create_volcano_plot(results_df, padj_threshold, log2fc_threshold):
    plt.scatter(results_df['log2FoldChange'], -np.log10(results_df['padj']), alpha=0.5)
    plt.axhline(-np.log10(padj_threshold), color='red', linestyle='--')
    plt.axvline(-log2fc_threshold, color='red', linestyle='--')
    plt.axvline(log2fc_threshold, color='red', linestyle='--')
    plt.xlabel('Log2 Fold Change')
    plt.ylabel('-Log10 Adjusted p-value')
    plt.title('Volcano Plot')

def create_ma_plot(results_df):
    plt.scatter(np.log10(results_df['baseMean']), results_df['log2FoldChange'], alpha=0.5)
    plt.xlabel('Log10 Mean Expression')
    plt.ylabel('Log2 Fold Change')
    plt.title('MA Plot')
    plt.axhline(y=0, color='r', linestyle='--')
    
def create_visualization_plots(results_df, significant_df, padj_threshold, log2fc_threshold):
    """
    Create visualization plots with proper data handling.
    """
    plt.figure(figsize=(15, 10))

    # Volcano plot
    plt.subplot(2, 2, 1)
    create_volcano_plot(results_df, padj_threshold, log2fc_threshold)

    # MA plot
    plt.subplot(2, 2, 2)
    create_ma_plot(results_df)

    # Save plots
    plt.tight_layout()
    plt.savefig('dexseq_analysis_plots.pdf')
    plt.close()

def save_results(results_df, significant_df, summary_stats):
    """
    Save analysis results to files.
    """
    results_df.to_csv("all_dexseq_results.csv", index=False)
    significant_df.to_csv("significant_dexseq_results.csv", index=False)
    summary_stats.to_csv("analysis_summary.csv", index=False)

In [108]:
def check_file_format(file_path):
    """
    Check if the file format is correct and print diagnostic information.
    """
    print(f"\nChecking file: {file_path}")
    try:
        with open(file_path, 'r') as f:
            first_lines = [next(f) for _ in range(5)]
        print("First 5 lines:")
        for line in first_lines:
            print(line.strip())
        return True
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return False

def format_count_file_for_dexseq(input_file, output_file):
    """
    Reformat count file to match DEXSeq expectations.
    """
    try:
        print(f"\nFormatting file: {input_file}")
        # Read the original file
        df = pd.read_csv(input_file, sep='\t', header=None, 
                        names=['feature_id', 'count'], quoting=3)
        
        # Split the feature_id into gene_id and exon_number
        def parse_feature_id(feature_id):
            if feature_id.startswith('*') or feature_id.startswith('_'):
                return feature_id, 'NA'
            try:
                parts = feature_id.replace('"', '').split(':')
                gene_id = parts[0]
                exon_number = parts[1] if len(parts) > 1 else 'NA'
                return gene_id, exon_number
            except:
                return feature_id, 'NA'
        
        # Apply parsing
        df[['gene_id', 'exon_number']] = df.apply(
            lambda x: pd.Series(parse_feature_id(x['feature_id'])), axis=1)
        
        # Write in DEXSeq expected format
        df.to_csv(output_file, sep='\t', 
                 columns=['gene_id', 'exon_number', 'count'],
                 header=False, index=False)
        
        # Verify the output
        print("\nVerifying output file format:")
        check_file_format(output_file)
        
        return True
    except Exception as e:
        print(f"Error formatting file {input_file}: {str(e)}")
        return False

def prepare_sample_info(sample_info_df):
    """
    Prepare sample information and print diagnostic information.
    """
    print("\nPreparing sample information:")
    print("Original sample info:")
    print(sample_info_df)
    
    # Filter for EDO and ND1 samples
    edo_nd1_samples = sample_info_df[sample_info_df['condition'].isin(['EDO', 'ND1'])].copy()
    edo_nd1_samples = edo_nd1_samples.reset_index(drop=True)
    
    # Sort by condition and replicate
    edo_nd1_samples = edo_nd1_samples.sort_values(['condition', 'replicate'])
    
    print("\nFiltered and sorted sample info:")
    print(edo_nd1_samples)
    
    # Create reformatted count files
    reformatted_files = []
    for idx, row in edo_nd1_samples.iterrows():
        original_file = row['count_file']
        reformatted_file = original_file + '.reformatted'
        if format_count_file_for_dexseq(original_file, reformatted_file):
            reformatted_files.append(reformatted_file)
        else:
            raise Exception(f"Failed to format count file: {original_file}")
    
    # Update count_file paths
    edo_nd1_samples['count_file'] = reformatted_files
    
    # Convert to categorical
    edo_nd1_samples['condition'] = pd.Categorical(edo_nd1_samples['condition'])
    edo_nd1_samples['replicate'] = pd.Categorical(edo_nd1_samples['replicate'])
    
    print("\nFinal sample information:")
    print(edo_nd1_samples)
    
    # Verify all files exist
    print("\nVerifying count files:")
    for file in edo_nd1_samples['count_file']:
        print(f"{file}: {os.path.exists(file)}")
    
    return edo_nd1_samples

def run_dexseq_analysis(sample_info_df, working_dir, padj_threshold=0.05, log2fc_threshold=1.0):
    """
    Run DEXSeq analysis with additional error checking.
    """
    print("\nStarting DEXSeq analysis...")
    print(f"Working directory: {working_dir}")
    
    # Verify GFF file
    gff_file = os.path.join(working_dir, "gencode.v31.basic.annotation.gff")
    if not os.path.exists(gff_file):
        raise FileNotFoundError(f"GFF file not found: {gff_file}")
    print(f"GFF file exists: {gff_file}")
    
    # Prepare samples
    try:
        edo_nd1_samples = prepare_sample_info(sample_info_df)
        
        # Import R packages
        print("\nImporting R packages...")
        dexseq = importr('DEXSeq')
        deseq2 = importr('DESeq2')
        
        # Convert sample information to R
        print("\nConverting sample information to R format...")
        r_sample_data = pandas2ri.py2rpy(edo_nd1_samples)
        
        # Create DEXSeqDataSet
        print("\nCreating DEXSeqDataSet...")
        print("Count files:")
        for file in edo_nd1_samples['count_file']:
            print(f"- {file}")
            
        dxd = dexseq.DEXSeqDataSetFromHTSeq(
            countfiles=ro.StrVector(edo_nd1_samples['count_file'].tolist()),
            sampleData=r_sample_data,
            design=r('~ sample + exon + condition:exon'),
            flattenedfile=gff_file
        )
        
        # Continue with analysis...
        print("Running DEXSeq pipeline...")
        
        return run_dexseq_pipeline(dxd, dexseq, padj_threshold, log2fc_threshold)
        
    except Exception as e:
        print(f"Error in DEXSeq analysis: {str(e)}")
        raise
        
def run_dexseq_pipeline(dxd, dexseq, padj_threshold, log2fc_threshold):
    """
    Run the DEXSeq pipeline steps with error checking.
    """
    try:
        print("Estimating size factors...")
        dxd = dexseq.estimateSizeFactors(dxd)
        
        print("Estimating dispersions...")
        r.pdf("dispersion_estimates.pdf")
        dxd = dexseq.estimateDispersions(dxd, quiet=False)
        r('dev.off()')
        
        print("Testing for differential exon usage...")
        dxd = dexseq.testForDEU(dxd)
        
        print("Estimating exon fold changes...")
        dxd = dexseq.estimateExonFoldChanges(dxd, fitExpToVar="condition")
        
        print("Getting results...")
        res = dexseq.DEXSeqResults(dxd)
        res_df = pandas2ri.rpy2py_dataframe(res)
        
        return process_results(res_df, padj_threshold, log2fc_threshold)
        
    except Exception as e:
        print(f"Error in DEXSeq pipeline: {str(e)}")
        raise


In [ ]:
# First, print current state
print("Current working directory:", os.getcwd())
print("\nSample information:")
print(sample_info)

try:
    results, significant_results, summary = run_dexseq_analysis(
        sample_info, 
        working_dir="/beegfs/scratch/ric.broccoli/kubacki.michal/PW_RNA_seq_deep",
        padj_threshold=0.05,
        log2fc_threshold=1.0
    )
    
except Exception as e:
    print(f"\nError in analysis: {str(e)}")
    print("\nFull error information:")
    import traceback
    traceback.print_exc()

In [ ]:
# Create DEXSeqDataSet with the filtered samples
dxd = dexseq.DEXSeqDataSetFromHTSeq(
    countfiles=ro.StrVector(edo_nd1_samples['count_file'].tolist()),
    sampleData=edo_nd1_samples,
    design=r('~ sample + exon + condition:exon'),
    flattenedfile=os.path.join(working_dir, "gencode.v31.basic.annotation.gff")
)

In [ ]:
if df is not None:
    # Log count distribution
    plt.subplot(132)
    plt.hist(np.log10(df['count'][df['count'] > 0]), bins=50)
    plt.title('Log10 Count Distribution')
    plt.xlabel('Log10(Counts)')
    plt.ylabel('Frequency')
    
    # Box plot
    plt.subplot(133)
    df_filtered = df[~df['feature_id'].str.startswith('*')]
    plt.boxplot(df_filtered['count'])
    plt.title('Count Boxplot\n(excluding special features)')
    plt.ylabel('Counts')
    
    plt.tight_layout()
    plt.savefig('count_distributions.pdf')
    plt.close()
    
    # Examine genes of interest
    genes_of_interest = ["SETD5", "NSD2", "POLE", "HTT", "PER3", "MED14"]
    examine_genes_of_interest(df, genes_of_interest)

In [ ]:




# Plot count distributions
print("\nPlotting count distributions...")
plot_count_distributions(count_files[:3])  # Examine first 3 files
plt.savefig('count_distributions.pdf')

In [ ]:
print("\nChecking for issues in files:")
for file_path in count_files:
    issues = check_file_issues(file_path)
    if issues:
        print(f"\n{os.path.basename(file_path)}:")
        for issue in issues:
            print(f"- {issue}")

# Save summary to file
with open('dexseq_counts_summary.txt', 'w') as f:
    f.write("DEXSeq Count Files Summary\n")
    f.write("=========================\n\n")
    f.write("File Structure Comparison:\n")
    f.write(structure_comparison.to_string())
    f.write("\n\nSample Statistics:\n")
    for file_path in count_files:
        df = pd.read_csv(file_path, sep='\t', header=None)
        f.write(f"\n{os.path.basename(file_path)}:\n")
        f.write(df[2].describe().to_string())

print("\nAnalysis complete. Check 'dexseq_counts_summary.txt' for detailed summary.")

In [24]:
# Ensure proper formatting for DEXSeq
edo_nd1_samples = edo_nd1_samples.reset_index(drop=True)
edo_nd1_samples['condition'] = pd.Categorical(edo_nd1_samples['condition'])
edo_nd1_samples['replicate'] = pd.Categorical(edo_nd1_samples['replicate'])

In [ ]:
# Create DEXSeqDataSet with the filtered samples
dxd = dexseq.DEXSeqDataSetFromHTSeq(
    countfiles=ro.StrVector(edo_nd1_samples['count_file'].tolist()),
    sampleData=edo_nd1_samples,
    design=r('~ sample + exon + condition:exon'),
    flattenedfile=os.path.join(working_dir, "gencode.v31.basic.annotation.gff")
)

# BAM FILES

In [3]:
# Define paths for RNA-seq data
bam_dir = os.path.join(data_dir, "Bam_file")
bam_files = [f for f in os.listdir(bam_dir) if f.endswith('.bam')]

In [ ]:
print(bam_files)

In [ ]:
# Separate patient and control files
EDO = [f for f in bam_files if 'EDO' in f]
PW1 = [f for f in bam_files if 'PW1' in f]
ND1 = [f for f in bam_files if 'ND1' in f]
print(EDO)
print(PW1)
print(ND1)

In [9]:
# Define the genes of interest from the pull-down experiment
genes_of_interest = ["SETD5", "NSD2", "POLE", "HTT", "PER3", "MED14"]

In [11]:
def extract_gene_coverage(bam_file, gene_regions):
    coverage = {}
    with pysam.AlignmentFile(bam_file, "rb") as bam:
        for gene, region in gene_regions.items():
            chrom, start, end = region
            coverage[gene] = bam.count_coverage(chrom, start, end)
    return coverage